### Import Packages

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from sklearn import feature_extraction, manifold
from sklearn.model_selection import train_test_split

# for model-building
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score

# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# hstack
from scipy.sparse import hstack

### Import Datasets

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# %cd /content/drive/My Drive/BT4222/Final Submission/datasets

/content/drive/My Drive/BT4222/Final Submission/datasets


In [ ]:
md = pd.read_csv('masterdata.csv')

### Data Pre-Processing

In [ ]:
md = md.drop(columns=['lang','label', 'text_', 'cleaned_text', 'category'])
md.head()

,rating,review_length,end_with_punc,clean_text,target,category_Books_5,category_Clothing_Shoes_and_Jewelry_5,category_Electronics_5,category_Home_and_Kitchen_5,category_Kindle_Store_5,category_Movies_and_TV_5,category_Pet_Supplies_5,category_Sports_and_Outdoors_5,category_Tools_and_Home_Improvement_5,category_Toys_and_Games_5
0,5.0,12,0,love well made sturdy comfortable love itvery ...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,16,0,love great upgrade original ive mine couple years,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,14,1,pillow saved back love look feel pillow,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,17,0,missing information use great product price,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,18,0,nice set good quality set two months,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Split train test sets
X = md.drop(columns=['target'], axis = 1)
y = pd.DataFrame(md.loc[:,'target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size = 0.2) #20% test

In [ ]:
X_train.head()

,rating,review_length,end_with_punc,clean_text,category_Books_5,category_Clothing_Shoes_and_Jewelry_5,category_Electronics_5,category_Home_and_Kitchen_5,category_Kindle_Store_5,category_Movies_and_TV_5,category_Pet_Supplies_5,category_Sports_and_Outdoors_5,category_Tools_and_Home_Improvement_5,category_Toys_and_Games_5
38545,1.0,18,1,div classasection aspacingsmall aspacingtopmin...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32566,5.0,22,1,got bunch kids christmas list perfect love add...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9173,1.0,34,1,cable arrived doa doesnt appear damaged way de...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17835,5.0,84,0,initial impression saw materials good saw made...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29948,5.0,73,1,seen first season reading first novel reeling ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (31220, 14)
X_test: (7805, 14)
y_train: (31220, 1)
y_test: (7805, 1)


In [ ]:
y_train_nb = y_train['target']
y_test_nb = y_test['target']

In [ ]:
print('y_train:', y_train_nb.shape)
print('y_test:', y_test_nb.shape)

y_train: (31220,)
y_test: (7805,)


### Naive Bayes (CountVect)

In [ ]:
def tokenize_test_nb_cv(X_train, y_train, X_test, y_test, vect):
    # create document-term matrices using the vectorizer
    
    # fit and transform training data
    global X_train_dtm_nb_cv
    global X_test_dtm_nb_cv
    X_train_dtm_nb_cv = vect.fit_transform(X_train['clean_text'])
    X_train_stack = X_train.drop(columns=['clean_text']).values
    X_train_dtm_nb_cv = np.hstack((X_train_dtm_nb_cv.toarray(), X_train_stack))
    # transform testing data (using fitted vocabulary) into a document-term matrix
    X_test_dtm_nb_cv = vect.transform(X_test['clean_text'])
    X_test_stack = X_test.drop(columns=['clean_text']).values
    X_test_dtm_nb_cv = np.hstack((X_test_dtm_nb_cv.toarray(), X_test_stack))
    
    # print the number of features that were generated
    print('Features: ', X_train_dtm_nb_cv.shape[1])
    
    # instantiate a Multinomial Naive Bayes model
    nb = MultinomialNB()
    # train the model
    %time nb.fit(X_train_dtm_nb_cv, y_train)
    # make class predictions for X_test_dtm_nb
    global y_pred_class_nb_cv
    global y_train_pred_nb_cv
    global y_pred_prob_nb_cv
    y_train_pred_nb_cv = nb.predict(X_train_dtm_nb_cv)
    y_pred_class_nb_cv = nb.predict(X_test_dtm_nb_cv)
    y_pred_prob_nb_cv = nb.predict_proba(X_test_dtm_nb_cv)[:, 1]
    
    # Get the training accuracy
    print('Training Accuracy using Count Vectorizer: ', metrics.accuracy_score(y_train, y_train_pred_nb_cv))
    # print the accuracy of its predictions
    print('Test Accuracy using Count Vectorizer: ', metrics.accuracy_score(y_test, y_pred_class_nb_cv))

In [ ]:
# instantiate vectorizer with the default parameters
vect_nb = CountVectorizer()

In [ ]:
tokenize_test_nb_cv(X_train, y_train_nb, X_test, y_test_nb, vect_nb )

In [ ]:
print('X_train_dtm:', X_train_dtm_nb_cv.shape)
print('X_test_dtm:', X_test_dtm_nb_cv.shape)

### Naive Bayes (TF-IDF)

In [ ]:
def tokenize_test_nb_tfidf(X_train, y_train, X_test, y_test, vect):
    # create document-term matrices using the vectorizer
    
    # fit and transform training data
    global X_train_dtm_nb_tfidf
    global X_test_dtm_nb_tfidf
    X_train_dtm_nb_tfidf = vect.fit_transform(X_train['clean_text'])
    X_train_stack = X_train.drop(columns=['clean_text']).values
    X_train_dtm_nb_tfidf = np.hstack((X_train_dtm_nb_tfidf.toarray(), X_train_stack))
    # transform testing data (using fitted vocabulary) into a document-term matrix
    X_test_dtm_nb_tfidf= vect.transform(X_test['clean_text'])
    X_test_stack = X_test.drop(columns=['clean_text']).values
    X_test_dtm_nb_tfidf = np.hstack((X_test_dtm_nb_tfidf.toarray(), X_test_stack))
    
    
    # print the number of features that were generated
    print("n_samples: %d, n_features: %d" % X_train_dtm_nb_tfidf.shape)
    
    # instantiate a Multinomial Naive Bayes model
    nb = MultinomialNB()
    # train the model
    %time nb.fit(X_train_dtm_nb_tfidf, y_train)
    # make class predictions for X_test_dtm_nb
    global y_train_pred_nb_tfidf
    global y_pred_class_nb_tfidf
    global y_pred_prob_nb_tfidf
    y_train_pred_nb_tfidf = nb.predict(X_train_dtm_nb_tfidf)
    y_pred_class_nb_tfidf = nb.predict(X_test_dtm_nb_tfidf)
    y_pred_prob_nb_tfidf = nb.predict_proba(X_test_dtm_nb_tfidf)[:, 1]
    
    # Get the training accuracy
    print('Training Accuracy using TF-IDF: ', metrics.accuracy_score(y_train, y_train_pred_nb_tfidf))
    # print the accuracy of its predictions
    print('Test Accuracy using TF-IDF: ', metrics.accuracy_score(y_test, y_pred_class_nb_tfidf))

In [ ]:
# instantiate vectorizer with the default parameters
nb_vect_tfidf = TfidfVectorizer()

In [ ]:
tokenize_test_nb_tfidf(X_train, y_train_nb, X_test, y_test_nb, nb_vect_tfidf)

n_samples: 31220, n_features: 42093
CPU times: user 8.76 s, sys: 13.9 s, total: 22.7 s
Wall time: 24.5 s
Training Accuracy using TF-IDF:  0.8866111467008329
Test Accuracy using TF-IDF:  0.8550928891736067


In [ ]:
print('X_train_dtm:', X_train_dtm_nb_tfidf.shape)
print('X_test_dtm:', X_test_dtm_nb_tfidf.shape)

X_train_dtm: (31220, 41939)
X_test_dtm: (7805, 41939)


### Comparing Accuracy of Different Vectorizers

In [ ]:
# Evaluation Metrics

print("No Feature Selection & No Hyperparameter Tuning")
print('')

print("Accuracy using Count Vectorizer: ", metrics.accuracy_score(y_test_nb, y_pred_class_nb_cv))
print("Accuracy using Tf-IDF: ", metrics.accuracy_score(y_test_nb, y_pred_class_nb_tfidf))
print ('')
print("Precision using Count Vectorizer: ", metrics.precision_score(y_test_nb, y_pred_class_nb_cv))
print("Precision using Tf-IDF: ", metrics.precision_score(y_test_nb, y_pred_class_nb_tfidf))
print('')
print("Recall using Count Vectorizer: ", metrics.recall_score(y_test_nb, y_pred_class_nb_cv))
print("Recall using Tf-IDF: ", metrics.recall_score(y_test_nb,  y_pred_class_nb_tfidf))
print('')
print("ROC_AUC using Count Vectorizer: ", metrics.roc_auc_score(y_test_nb, y_pred_prob_nb_cv))
print("ROC_AUC using Tf-IDF: ", metrics.roc_auc_score(y_test_nb, y_pred_prob_nb_tfidf))
print ('')
print("F1_Score using Count Vectorizer: ", metrics.f1_score(y_test_nb, y_pred_class_nb_cv))
print("F1_Score using Tf-IDF: ", metrics.f1_score(y_test_nb, y_pred_class_nb_tfidf))


No Feature Selection & No Hyperparameter Tuning

Accuracy using Count Vectorizer:  0.8547085201793722
Accuracy using Tf-IDF:  0.8572709801409353

Precision using Count Vectorizer:  0.8102123813827383
Precision using Tf-IDF:  0.8244955419990615

Recall using Count Vectorizer:  0.9242268041237114
Recall using Tf-IDF:  0.9056701030927835

ROC_AUC using Count Vectorizer:  0.9452124236653752
ROC_AUC using Tf-IDF:  0.9415108674239936

F1_Score using Count Vectorizer:  0.8634721887791956
F1_Score using Tf-IDF:  0.8631785802014247


### Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

#### CountVectorizer

In [ ]:
# select at least 20000 features that scored the best (feature extraction)
selector_nb = SelectKBest(f_classif, k=min(20000, X_train_dtm_nb_cv.shape[1]))
selector_nb.fit(X_train_dtm_nb_cv, y_train_nb)

# feature scores
print(selector_nb.scores_)

# final X_train_dtm for NB
X_train_nb_fs_cv = selector_nb.transform(X_train_dtm_nb_cv) 
# final X_test_dtm for NB
X_test_nb_fs_cv = selector_nb.transform(X_test_dtm_nb_cv) 

[2.28341170e+00 7.22328485e+00 9.83985260e-01 ... 2.17676233e-01
 2.45572040e-02 5.10140675e-03]


In [ ]:
print(X_train_nb_fs_cv.shape)
print(X_train_dtm_nb_cv.shape)

(31220, 20000)
(31220, 42093)


In [ ]:
# instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()
# train the model
%time nb.fit(X_train_nb_fs_cv, y_train_nb)
# make class predictions for X_test_dtm_nb
y_train_pred_fs_cv = nb.predict(X_train_nb_fs_cv)
y_pred_class_fs_cv = nb.predict(X_test_nb_fs_cv)
y_pred_prob_fs_cv = nb.predict_proba(X_test_nb_fs_cv)[:, 1]
    
# Get the training accuracy
print('Training Accuracy using Count Vectorizer: ', metrics.accuracy_score(y_train_nb, y_train_pred_fs_cv))
# print the accuracy of its predictions
print('Test Accuracy using Count Vectorizer: ', metrics.accuracy_score(y_test_nb, y_pred_class_fs_cv))

CPU times: user 5.13 s, sys: 6.01 s, total: 11.1 s
Wall time: 13.4 s
Training Accuracy using Count Vectorizer:  0.8869634849455478
Test Accuracy using Count Vectorizer:  0.8652146060217809


#### Tf-IDF

In [ ]:
# select at least 20000 features that scored the best (feature extraction)
selector_nb_tfidf = SelectKBest(f_classif, k=min(20000, X_train_dtm_nb_tfidf.shape[1]))
selector_nb_tfidf.fit(X_train_dtm_nb_tfidf, y_train_nb)

# feature scores
print(selector_nb_tfidf.scores_)

# final X_train_dtm for NB
X_train_nb_fs_tfidf = selector_nb_tfidf.transform(X_train_dtm_nb_tfidf) 
# final X_test_dtm for NB
X_test_nb_fs_tfidf = selector_nb_tfidf.transform(X_test_dtm_nb_tfidf) 

[1.06991239e+00 5.21647383e+00 9.83985260e-01 ... 2.17676233e-01
 2.45572040e-02 5.10140675e-03]


In [ ]:
print(X_train_nb_fs_tfidf.shape)
# print(X_train_dtm_nb_cv.shape)

In [ ]:
# instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()
# train the model
%time nb.fit(X_train_nb_fs_tfidf, y_train_nb)
# make class predictions for X_test_dtm_nb
y_train_pred_fs_tfidf = nb.predict(X_train_nb_fs_tfidf)
y_pred_class_fs_tfidf = nb.predict(X_test_nb_fs_tfidf)
y_pred_prob_fs_tfidf = nb.predict_proba(X_test_nb_fs_tfidf)[:, 1]
    
# Get the training accuracy
print('Training Accuracy using Tf-IDF: ', metrics.accuracy_score(y_train_nb, y_train_pred_fs_tfidf))
# print the accuracy of its predictions
print('Test Accuracy using Tf-IDF: ', metrics.accuracy_score(y_test_nb, y_pred_class_fs_tfidf))

CPU times: user 4.17 s, sys: 4.38 s, total: 8.56 s
Wall time: 9.64 s
Training Accuracy using Tf-IDF:  0.923542600896861
Test Accuracy using Tf-IDF:  0.8969891095451633


In [ ]:
# Evaluation Metrics
print("After Feature Selection")
print('')
print("Accuracy using Count Vectorizer: ", metrics.accuracy_score(y_test_nb, y_pred_class_fs_cv))
print("Accuracy using Tf-IDF: ", metrics.accuracy_score(y_test_nb, y_pred_class_fs_tfidf))
print ('')
print("Precision using Count Vectorizer: ", metrics.precision_score(y_test_nb, y_pred_class_fs_cv))
print("Precision using Tf-IDF: ", metrics.precision_score(y_test_nb, y_pred_class_fs_tfidf))
print('')
print("Recall using Count Vectorizer: ", metrics.recall_score(y_test_nb, y_pred_class_fs_cv))
print("Recall using Tf-IDF: ", metrics.recall_score(y_test_nb, y_pred_class_fs_tfidf))
print('')
print("ROC_AUC using Count Vectorizer: ", metrics.roc_auc_score(y_test_nb, y_pred_prob_fs_cv))
print("ROC_AUC using Tf-IDF: ", metrics.roc_auc_score(y_test_nb, y_pred_prob_fs_tfidf))
print ('')
print("F1_Score using Count Vectorizer: ", metrics.f1_score(y_test_nb, y_pred_class_fs_cv))
print("F1_Score using Tf-IDF: ", metrics.f1_score(y_test_nb, y_pred_class_fs_cv))

After Feature Selection

Accuracy using Count Vectorizer:  0.8652146060217809
Accuracy using Tf-IDF:  0.8969891095451633

Precision using Count Vectorizer:  0.8482741494909362
Precision using Tf-IDF:  0.9045852107076597

Recall using Count Vectorizer:  0.8856624319419237
Recall using Tf-IDF:  0.8848846253564947

ROC_AUC using Count Vectorizer:  0.943136782843809
ROC_AUC using Tf-IDF:  0.9651127740743749

F1_Score using Count Vectorizer:  0.8665651953323185
F1_Score using Tf-IDF:  0.8665651953323185


### Hyperparameter Tuning

#### alpha=1, ngram_range=(1,2)

In [ ]:
lst = []

for x in range (5, 11):
    # instantiate vectorizer
    nb_vect_optimal = TfidfVectorizer(min_df=x, ngram_range=(1,2))
    
    X_train_dtm_nb_optimal= nb_vect_optimal.fit_transform(X_train['clean_text'])
    X_train_stack = X_train.drop(columns=['clean_text']).values
    X_train_dtm_nb_optimal= np.hstack((X_train_dtm_nb_optimal.toarray(), X_train_stack))
    
    X_test_dtm_nb_optimal = nb_vect_optimal.transform(X_test['clean_text'])
    X_test_stack = X_test.drop(columns=['clean_text']).values
    X_test_dtm_nb_optimal = np.hstack((X_test_dtm_nb_optimal.toarray(), X_test_stack))
    
    # select at least 20000 features that scored the best (feature extraction)
    selector_nb_optimal = SelectKBest(f_classif, k=min(20000, X_train_dtm_nb_optimal.shape[1]))
    selector_nb_optimal.fit(X_train_dtm_nb_optimal, y_train_nb)
    
    # final X_train_dtm for NB
    X_train_nb_fs_optimal = selector_nb_optimal.transform(X_train_dtm_nb_optimal) 
    # final X_test_dtm for NB
    X_test_nb_fs_optimal = selector_nb_optimal.transform(X_test_dtm_nb_optimal) 

    # instantiate a Multinomial Naive Bayes model with optimal parameters
    nb_optimal = MultinomialNB(alpha=1)
    
    # train the model
    %time nb_optimal.fit(X_train_nb_fs_optimal, y_train_nb)
    
    # make class predictions for X_test_dtm_nb
    y_train_pred_optimal = nb_optimal.predict(X_train_nb_fs_optimal)
    y_pred_class_optimal = nb_optimal.predict(X_test_nb_fs_optimal)
    y_pred_prob_optimal = nb_optimal.predict_proba(X_test_nb_fs_optimal)[:, 1]
    
    # Get the test accuracy
    lst.append(tuple([x, metrics.accuracy_score(y_test_nb, y_pred_class_optimal)]))

CPU times: user 5.17 s, sys: 7.12 s, total: 12.3 s
Wall time: 21.9 s
CPU times: user 5 s, sys: 6.5 s, total: 11.5 s
Wall time: 20.8 s
CPU times: user 5.98 s, sys: 7.1 s, total: 13.1 s
Wall time: 19.7 s
CPU times: user 4.17 s, sys: 4.13 s, total: 8.31 s
Wall time: 8.31 s
CPU times: user 2.42 s, sys: 1.92 s, total: 4.34 s
Wall time: 4.7 s
CPU times: user 3.38 s, sys: 3.63 s, total: 7.02 s
Wall time: 8.56 s


In [ ]:
lst

[(5, 0.9037796284433056),
 (6, 0.903267136450993),
 (7, 0.9010890454836643),
 (8, 0.8707238949391416),
 (9, 0.8695707879564382),
 (10, 0.8695707879564382)]

In [ ]:
print("After Feature Selection and Hyperparameter Tuning")
print('')

for i in lst:
    print("Accuracy using ngram_range=(1,2), alpha=1, min_df =",i[0], ": ", i[1]*100)

After Feature Selection and Hyperparameter Tuning

Accuracy using ngram_range=(1,2), alpha=1, min_df = 5 :  90.37796284433057
Accuracy using ngram_range=(1,2), alpha=1, min_df = 6 :  90.3267136450993
Accuracy using ngram_range=(1,2), alpha=1, min_df = 7 :  90.10890454836643
Accuracy using ngram_range=(1,2), alpha=1, min_df = 8 :  87.07238949391416
Accuracy using ngram_range=(1,2), alpha=1, min_df = 9 :  86.95707879564381
Accuracy using ngram_range=(1,2), alpha=1, min_df = 10 :  86.95707879564381


In [ ]:
lst_bigram = []

for x in range (5, 11):
    # instantiate vectorizer
    nb_vect_optimal = TfidfVectorizer(min_df=x, ngram_range=(2,2))
    
    X_train_dtm_nb_optimal= nb_vect_optimal.fit_transform(X_train['clean_text'])
    X_train_stack = X_train.drop(columns=['clean_text']).values
    X_train_dtm_nb_optimal= np.hstack((X_train_dtm_nb_optimal.toarray(), X_train_stack))
    
    X_test_dtm_nb_optimal = nb_vect_optimal.transform(X_test['clean_text'])
    X_test_stack = X_test.drop(columns=['clean_text']).values
    X_test_dtm_nb_optimal = np.hstack((X_test_dtm_nb_optimal.toarray(), X_test_stack))
    
    # select at least 20000 features that scored the best (feature extraction)
    selector_nb_optimal = SelectKBest(f_classif, k=min(20000, X_train_dtm_nb_optimal.shape[1]))
    selector_nb_optimal.fit(X_train_dtm_nb_optimal, y_train_nb)
    
    # final X_train_dtm for NB
    X_train_nb_fs_optimal = selector_nb_optimal.transform(X_train_dtm_nb_optimal) 
    # final X_test_dtm for NB
    X_test_nb_fs_optimal = selector_nb_optimal.transform(X_test_dtm_nb_optimal) 

    # instantiate a Multinomial Naive Bayes model with optimal parameters
    nb_optimal = MultinomialNB(alpha=1)
    
    # train the model
    %time nb_optimal.fit(X_train_nb_fs_optimal, y_train_nb)
    
    # make class predictions for X_test_dtm_nb
    y_train_pred_optimal = nb_optimal.predict(X_train_nb_fs_optimal)
    y_pred_class_optimal = nb_optimal.predict(X_test_nb_fs_optimal)
    y_pred_prob_optimal = nb_optimal.predict_proba(X_test_nb_fs_optimal)[:, 1]
    
    # Get the test accuracy
    lst_bigram.append(tuple([x, metrics.accuracy_score(y_test_nb, y_pred_class_optimal)]))

CPU times: user 5.39 s, sys: 6.25 s, total: 11.6 s
Wall time: 12.8 s
CPU times: user 2.36 s, sys: 1.81 s, total: 4.17 s
Wall time: 4.4 s
CPU times: user 2.96 s, sys: 3.3 s, total: 6.26 s
Wall time: 10.1 s
CPU times: user 2.55 s, sys: 2.92 s, total: 5.48 s
Wall time: 10.7 s
CPU times: user 1.23 s, sys: 870 ms, total: 2.1 s
Wall time: 2.05 s
CPU times: user 1.1 s, sys: 760 ms, total: 1.86 s
Wall time: 1.91 s


In [ ]:
lst_bigram

[(5, 0.8524023062139654),
 (6, 0.8430493273542601),
 (7, 0.8402306213965407),
 (8, 0.8392056374119155),
 (9, 0.8366431774503523),
 (10, 0.8344650864830238)]

In [ ]:
print("After Feature Selection and Hyperparameter Tuning")
print('')

for i in lst_bigram:
    print("Accuracy using ngram_range=(2,2), alpha=1, min_df =",i[0], ": ", i[1]*100)

After Feature Selection and Hyperparameter Tuning

Accuracy using ngram_range=(2,2), alpha=1, min_df = 5 :  85.24023062139653
Accuracy using ngram_range=(2,2), alpha=1, min_df = 6 :  84.30493273542601
Accuracy using ngram_range=(2,2), alpha=1, min_df = 7 :  84.02306213965407
Accuracy using ngram_range=(2,2), alpha=1, min_df = 8 :  83.92056374119154
Accuracy using ngram_range=(2,2), alpha=1, min_df = 9 :  83.66431774503523
Accuracy using ngram_range=(2,2), alpha=1, min_df = 10 :  83.44650864830237


### Hypothesis 3: Running the best model

In [ ]:
nb_vect_best = TfidfVectorizer(min_df=5, ngram_range=(1,2))

In [ ]:
X_train_dtm_nb_best= nb_vect_best.fit_transform(X_train['clean_text'])
X_train_stack = X_train.drop(columns=['clean_text']).values
X_train_dtm_nb_best= np.hstack((X_train_dtm_nb_best.toarray(), X_train_stack))
    
X_test_dtm_nb_best = nb_vect_best.transform(X_test['clean_text'])
X_test_stack = X_test.drop(columns=['clean_text']).values
X_test_dtm_nb_best = np.hstack((X_test_dtm_nb_best.toarray(), X_test_stack))

In [ ]:
# select at least 20000 features that scored the best (feature extraction)
selector_nb_best = SelectKBest(f_classif, k=min(20000, X_train_dtm_nb_best.shape[1]))
selector_nb_best.fit(X_train_dtm_nb_best, y_train_nb)
    
# final X_train_dtm for NB
X_train_nb_fs_best = selector_nb_best.transform(X_train_dtm_nb_best) 
# final X_test_dtm for NB
X_test_nb_fs_best = selector_nb_best.transform(X_test_dtm_nb_best) 

In [ ]:
nb_best = MultinomialNB(alpha=1)
    
# train the model
%time nb_best.fit(X_train_nb_fs_best, y_train_nb)
    
# make class predictions 
y_train_pred_best = nb_best.predict(X_train_nb_fs_best)
y_pred_class_best = nb_best.predict(X_test_nb_fs_best)
y_pred_prob_best = nb_best.predict_proba(X_test_nb_fs_best)[:, 1]

CPU times: user 4.68 s, sys: 5.68 s, total: 10.4 s
Wall time: 15.3 s


In [ ]:
print('Best Model')
print('')
print("Accuracy: ", metrics.accuracy_score(y_test_nb, y_pred_class_best))
print ('')
print("Precision: ", metrics.precision_score(y_test_nb, y_pred_class_best))
print('')
print("Recall: ", metrics.recall_score(y_test_nb, y_pred_class_best))
print('')
print("ROC_AUC: ", metrics.roc_auc_score(y_test_nb, y_pred_prob_best))
print ('')
print("F1_Score: ", metrics.f1_score(y_test_nb, y_pred_class_best))

Best Model

Accuracy:  0.9083920563741191

Precision:  0.9448550312677658

Recall:  0.8644993498049415

ROC_AUC:  0.9736363636363636

F1_Score:  0.9028928425913351
